In [ ]:
import datetime
from typing import List
import pandas as pd

class Claim:
    def __init__(self, claim_id: str, policy_id: str, claim_type: str, amount: float,
                 claim_date: datetime.date, description: str, is_fraudulent: bool):
        self.claim_id = claim_id
        self.policy_id = policy_id
        self.claim_type = claim_type
        self.amount = amount
        self.claim_date = claim_date
        self.description = description
        self.is_fraudulent = is_fraudulent
        self.previous_claims = 0
        self.has_inconsistencies = False
        self.has_witnesses = True

    def __str__(self):
        return f"{self.claim_id} ({self.claim_type}): ${self.amount:.2f} - {self.description}"

class FraudResult:
    def __init__(self, claim: Claim, fraud_score: float, risk_level: str, red_flags: List[str]):
        self.claim = claim
        self.fraud_score = fraud_score
        self.risk_level = risk_level
        self.red_flags = red_flags

    def __str__(self):
        return f"{self.claim} | Score: {self.fraud_score:.1f} ({self.risk_level}) | Flags: {self.red_flags}"

class FraudDetectionSystem:
    HIGH_RISK = 70.0
    MEDIUM_RISK = 40.0

    def load_data(self):
        data = {
            'claim_id': ['101', '102', '103', '104', '105', '106', '107', '108', '109', '110'],
            'amount': [2500, 50000, 12000, 800, 35000, 1500, 9000, 3000, 20000, 5000],
            'type': ['Auto', 'Home', 'Health', 'Auto', 'Auto', 'Home', 'Health', 'Auto', 'Home', 'Health'],
            'days_to_report': [5, 1, 2, 10, 1, 3, 7, 2, 1, 4],
            'previous_claims': [0, 3, 1, 0, 5, 0, 2, 1, 4, 0],
            'has_witness': [1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
            'document_issues': [0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
            'fraud': [0, 1, 0, 0, 1, 0, 0, 0, 1, 0]
        }
        df = pd.DataFrame(data)
        claims = []
        for _, row in df.iterrows():
            claim = Claim(
                claim_id=row['claim_id'],
                policy_id=f"P{row['claim_id']}",
                claim_type=row['type'],
                amount=row['amount'],
                claim_date=datetime.date.today() - datetime.timedelta(days=row['days_to_report']),
                description="Sample claim",
                is_fraudulent=bool(row['fraud'])
            )
            claim.previous_claims = row['previous_claims']
            claim.has_witnesses = bool(row['has_witness'])
            claim.has_inconsistencies = bool(row['document_issues'])
            claims.append(claim)
        return claims

    def analyze_claims(self, claims: List[Claim]) -> List[FraudResult]:
        results = []
        for claim in claims:
            score = self._calculate_score(claim)
            risk_level = self._get_risk_level(score)
            flags = self._find_red_flags(claim)
            results.append(FraudResult(claim, score, risk_level, flags))
        return results

    def _calculate_score(self, claim: Claim) -> float:
        score = 0
        if claim.amount > 30000:
            score += 30
        elif claim.amount > 10000:
            score += 15
        score += min(claim.previous_claims * 10, 30)
        if claim.has_inconsistencies:
            score += 25
        if not claim.has_witnesses:
            score += 15
        if claim.claim_date > datetime.date.today() - datetime.timedelta(days=30):
            score += 10
        return min(score, 100)

    def _get_risk_level(self, score: float) -> str:
        if score >= self.HIGH_RISK:
            return "High"
        if score >= self.MEDIUM_RISK:
            return "Medium"
        return "Low"

    def _find_red_flags(self, claim: Claim) -> List[str]:
        flags = []
        if claim.amount > 30000:
            flags.append("High amount")
        if claim.previous_claims > 2:
            flags.append("Multiple claims")
        if claim.has_inconsistencies:
            flags.append("Inconsistent docs")
        if not claim.has_witnesses:
            flags.append("No witnesses")
        if claim.claim_date > datetime.date.today() - datetime.timedelta(days=7):
            flags.append("Recent claim")
        return flags

    def get_high_priority_claims(self, results: List[FraudResult]) -> List[FraudResult]:
        high_priority = [r for r in results if r.risk_level in ["High", "Medium"]]
        high_priority.sort(key=lambda x: x.fraud_score, reverse=True)
        return high_priority

def main():
    fds = FraudDetectionSystem()
    claims = fds.load_data()
    results = fds.analyze_claims(claims)

    print("Fraud Detection Results:")
    print("========================")
    for result in results:
        print(result)

    print("\nHigh Priority Claims for Investigation:")
    print("========================================")
    for result in fds.get_high_priority_claims(results):
        print(result)

if __name__ == "__main__":
    main()

Fraud Detection Results:
101 (Auto): $2500.00 - Sample claim | Score: 10.0 (Low) | Flags: ['Recent claim']
102 (Home): $50000.00 - Sample claim | Score: 100.0 (High) | Flags: ['High amount', 'Multiple claims', 'Inconsistent docs', 'No witnesses', 'Recent claim']
103 (Health): $12000.00 - Sample claim | Score: 35.0 (Low) | Flags: ['Recent claim']
104 (Auto): $800.00 - Sample claim | Score: 10.0 (Low) | Flags: []
105 (Auto): $35000.00 - Sample claim | Score: 100.0 (High) | Flags: ['High amount', 'Multiple claims', 'Inconsistent docs', 'No witnesses', 'Recent claim']
106 (Home): $1500.00 - Sample claim | Score: 10.0 (Low) | Flags: ['Recent claim']
107 (Health): $9000.00 - Sample claim | Score: 30.0 (Low) | Flags: []
108 (Auto): $3000.00 - Sample claim | Score: 20.0 (Low) | Flags: ['Recent claim']
109 (Home): $20000.00 - Sample claim | Score: 95.0 (High) | Flags: ['Multiple claims', 'Inconsistent docs', 'No witnesses', 'Recent claim']
110 (Health): $5000.00 - Sample claim | Score: 10.0 (Lo